<a href="https://colab.research.google.com/github/JavaFXpert/ReActTextWorldLangChain/blob/main/ReActTextWorldPublic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playing a TextWorld game with a LangChain ReAct agent
Notebook for playing a TextWorld game with a LangChain ReAct agent.

## Prerequisite
Install TextWorld as described in the [README.md](https://github.com/microsoft/TextWorld#readme). Most of the time, a simple `pip install` should work.

<span style="color:red">_*Notice here that TextWorld is installed with the `[vis]` extras to enable visual rendering of the world._</span>

In [ ]:
# For Google Colab, first install
!apt install chromium-chromedriver
!pip install selenium==3.12.0
# then
!pip install textworld[vis]

In [ ]:
!pip install langchain

In [ ]:
!pip install openai

#Generate TextWorld Game#

In [ ]:
import textworld
!tw-make custom --world-size 2 --quest-length 3 --nb-objects 10 --output tw_games/game.ulx -f -v --seed 456

In [ ]:
# Let the environment know what information we want as part of the game state.
infos = textworld.EnvInfos(
    feedback=True,    # Response from the game after typing a text command.
    description=True, # Text describing the room the player is currently in.
    inventory=True    # Text describing the player's inventory.
)

In [ ]:
# Get a reference to the game environment.
TW_ENV = textworld.start('tw_games/game.ulx', infos)

#LangChain ReAct stuff#

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain import OpenAI

In [ ]:
EXAMPLES = [
    """Setup: You are now playing a fast paced round of TextWorld! Here is your task for
today. First of all, you could, like, try to travel east. After that, take the
binder from the locker. With the binder, place the binder on the mantelpiece.
Alright, thanks!

-= Vault =-
You've just walked into a vault. You begin to take stock of what's here.

An open safe is here. What a letdown! The safe is empty! You make out a shelf.
But the thing hasn't got anything on it. What, you think everything in TextWorld
should have stuff on it?

You don't like doors? Why not try going east, that entranceway is unguarded.

Thought 1: I need to travel east
Action 1: Play[go east]
Observation 1: -= Office =-
You arrive in an office. An ordinary one.

You can make out a locker. The locker contains a binder. You see a case. The
case is empty, what a horrible day! You lean against the wall, inadvertently
pressing a secret button. The wall opens up to reveal a mantelpiece. You wonder
idly who left that here. The mantelpiece is standard. The mantelpiece appears to
be empty. If you haven't noticed it already, there seems to be something there
by the wall, it's a table. Unfortunately, there isn't a thing on it. Hm. Oh well
There is an exit to the west. Don't worry, it is unguarded.

Thought 2: I need to take the binder from the locker
Action 2: Play[take binder]
Observation 2: You take the binder from the locker.

Thought 3: I need to place the binder on the mantelpiece
Action 3: Play[put binder on mantelpiece]

Observation 3: You put the binder on the mantelpiece.
Your score has just gone up by one point.
*** The End ***
Thought 4: The End has occurred
Action 4: Finish[yes]

"""
]
SUFFIX = """\n\nSetup: {input}"""

PROMPT = PromptTemplate.from_examples(EXAMPLES, SUFFIX, ["input"])


In [ ]:
import re
from typing import Any, ClassVar, Dict, List, Optional, Tuple

from pydantic import BaseModel

from langchain.agents.agent import Agent
from langchain.agents.tools import Tool
from langchain.chains.llm import LLMChain
from langchain.docstore.base import Docstore
from langchain.docstore.document import Document
from langchain.llms.base import LLM
from langchain.prompts.base import BasePromptTemplate
from langchain.input import ChainedInput, get_color_mapping
from langchain.agents.react.base import ReActDocstoreAgent

from textworld.core import Environment, GameState

class ReActTextWorldAgent(ReActDocstoreAgent, BaseModel):
    """Agent for the ReAct TextWorld chain."""

    prompt: ClassVar[BasePromptTemplate] = PROMPT

    i: int = 1

    @classmethod
    def _validate_tools(cls, tools: List[Tool]) -> None:
        if len(tools) != 1:
            raise ValueError(f"Exactly one tool must be specified, but got {tools}")
        tool_names = {tool.name for tool in tools}
        if tool_names != {"Play"}:
            raise ValueError(
                f"Tool name should be Play, got {tool_names}"
            )


In [ ]:
import sys
class HiddenPrints:
    """Context manager to hide prints."""

    def __enter__(self) -> None:
        """Open file to pipe stdout to."""
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")

    def __exit__(self, *_: Any) -> None:
        """Close file that stdout was piped to."""
        sys.stdout.close()
        sys.stdout = self._original_stdout

class TextWorldInteract:
    """Class to interact with a TextWorld game."""

    def __init__(self, textworld_env: Environment):
        """Initialize with the TextWorld game environment."""
        self.textworld_env = textworld_env

    # def init(self) -> str:
    #     """Initialize the game and return initial render."""
    #     if self.textworld_env is None:
    #         raise ValueError("TextWorld game environment not set")
    #     game_state = self.textworld_env.reset()    
    #     render_str = game_state.objective + "\n\n" + game_state.description
    #     return render_str   

    def play(self, command) -> str:
        """Process a command and render the result."""
        if self.textworld_env is None:
            raise ValueError("TextWorld game environment not set")
        with HiddenPrints():
          game_state, reward, done = self.textworld_env.step(command)    
        return game_state["feedback"]       


In [ ]:
textworld_tool=TextWorldInteract(TW_ENV)
tools = [
    # TODO: Try to use this tool to initialize a game and produce the first render, rather than supplying the first render to the run method
    # Tool(
    #     name = "Init",
    #     func = textworld_tool.init,
    #     description="Initializes the TextWorld game environment requests initial game text"
    # ),
    Tool(
        name = "Play",
        func = textworld_tool.play,
        description="useful for interacting with a TextWorld game environment"
    )
]

llm = OpenAI(temperature=0)

In [ ]:
react = ReActTextWorldAgent.from_llm_and_tools(llm, tools)
react.verbose = True

In [ ]:
print(PROMPT.template)

In [ ]:
game_state = TW_ENV.reset()
init_text = game_state.objective + "\n\n" + game_state.description
react.run(init_text)

#Play game manually#

In [ ]:
game_state = TW_ENV.reset()

In [ ]:
TW_ENV.render()

In [ ]:
# game_state, reward, done = TW_ENV.step("go south")
# TW_ENV.render()